In [34]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator
import pandas as pd



path  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/MSR-Cambridge/Part2" 

names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']


#['TimeStamp','Response','IOType','LUN','ByteOffset','Size']


all_files = glob.glob(os.path.join(path, "src1_1.csv"))
print(all_files)




['/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/MSR-Cambridge/Part2/src1_1.csv']


In [37]:
df = pd.read_csv(all_files[0],engine='python',skiprows =2,header=None,na_values=['-1'], index_col=False) 
names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']
df.columns = names
print (len(df))

45746221


In [38]:
from collections import Counter
print(len(Counter(df['IOSize'])))
print(len(Counter(df['ByteOffset'])))

128
6170590


In [39]:

df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
df = df.drop(df.index[-1])




In [40]:
#Sorting df by TimeStamp

df = df.sort_values(by=['TimeStamp'])
df.reset_index(inplace=True, drop=True)
print(df.dtypes)
print(len(df))

TimeStamp             int64
Host_Name            object
DiskNumber          float64
Operation_Type       object
ByteOffset            int64
IOSize                int64
Response_Time         int64
DiskNum               int64
ByteOffset_Delta    float64
dtype: object
45746220


In [41]:
df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)

a = df['ByteOffset_Delta'].unique().tolist()
operation_id_map = {}
for i,id in enumerate(a): operation_id_map[id] = i 
df['ByteOffset_Delta_class'] = df['ByteOffset_Delta'].map(lambda x: operation_id_map[x])


In [42]:
df.head(5)

,TimeStamp,Host_Name,DiskNumber,Operation_Type,ByteOffset,IOSize,Response_Time,DiskNum,ByteOffset_Delta,ByteOffset_Delta_class
0,128166372013535022,src1,NaN,Write,3337596928,65536,3543,1,-9.871325e+10,0
1,128166372013536418,src1,NaN,Write,102050844672,4096,2148,1,9.887073e+10,1
2,128166372013537059,src1,NaN,Write,3180109824,8192,1507,1,-8.192000e+03,2
3,128166372021660589,src1,NaN,Write,3180118016,28672,2925,1,2.598093e+07,3
4,128166372021662159,src1,NaN,Write,3154137088,4096,1355,1,1.228800e+04,4


In [43]:
from collections import Counter
x = Counter(df['ByteOffset_Delta_class'])
vals = {}
vals =  x.most_common(1000)
bo_list = []

for x in vals:
    bo_list.append(x[0])
        
count = 0
label_list = []

while (count < len(df)):
    x = df['ByteOffset_Delta_class'].iloc[count]
    if x in bo_list:
        label_list.append(x)
    else:
        label_list.append(999999)
    count= count + 1
    
ByteOffset_Delta_class_backup  = df['ByteOffset_Delta_class'] 
df['ByteOffset_Delta_class']  = label_list
print(len(Counter(df['ByteOffset_Delta_class'])))
    
a = df['ByteOffset_Delta_class'].unique().tolist()
bo_map = {}
for i,id in enumerate(a): bo_map[id] = i 
df['ByteOffset_Delta_Class_1001'] = df['ByteOffset_Delta_class'].map(lambda x: bo_map[x])
    
label_list = df['ByteOffset_Delta_Class_1001'] 
    
df['ByteOffset_Delta_Class_1001']  = label_list
    

1001


In [44]:
count  = 0
size_dec_norm = []
size_dec = df['IOSize'].tolist()
min_size = min(size_dec)
max_size = max(size_dec)
while (count < len(size_dec)):
    tmp = (float(size_dec[count])- min_size)/(max_size - min_size)
    count  = count +1
    size_dec_norm.append(tmp)
    
df['Size'] = size_dec_norm

In [45]:
df['Size'] = size_dec_norm

In [46]:
len(Counter(df['ByteOffset_Delta_Class_1001']))

1001

In [47]:
len(Counter(df['Size']))

128

In [48]:
df.dtypes

TimeStamp                        int64
Host_Name                       object
DiskNumber                     float64
Operation_Type                  object
ByteOffset                       int64
IOSize                           int64
Response_Time                    int64
DiskNum                          int64
ByteOffset_Delta               float64
ByteOffset_Delta_class           int64
ByteOffset_Delta_Class_1001      int64
Size                           float64
dtype: object

In [49]:
df.head(5)

,TimeStamp,Host_Name,DiskNumber,Operation_Type,ByteOffset,IOSize,Response_Time,DiskNum,ByteOffset_Delta,ByteOffset_Delta_class,ByteOffset_Delta_Class_1001,Size
0,128166372013535022,src1,NaN,Write,3337596928,65536,3543,1,-9.871325e+10,999999,0,1.000000
1,128166372013536418,src1,NaN,Write,102050844672,4096,2148,1,9.887073e+10,999999,0,0.055118
2,128166372013537059,src1,NaN,Write,3180109824,8192,1507,1,-8.192000e+03,2,1,0.118110
3,128166372021660589,src1,NaN,Write,3180118016,28672,2925,1,2.598093e+07,999999,0,0.433071
4,128166372021662159,src1,NaN,Write,3154137088,4096,1355,1,1.228800e+04,4,2,0.055118


In [50]:
# Drop unnecessary coloumns

print("Before drop: {}".format(df.columns))
df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]], axis=1,inplace=True) 


Before drop: Index(['TimeStamp', 'Host_Name', 'DiskNumber', 'Operation_Type', 'ByteOffset',
       'IOSize', 'Response_Time', 'DiskNum', 'ByteOffset_Delta',
       'ByteOffset_Delta_class', 'ByteOffset_Delta_Class_1001', 'Size'],
      dtype='object')


In [51]:
df.dtypes

ByteOffset_Delta_Class_1001      int64
Size                           float64
dtype: object

In [52]:
from collections import Counter
print(len(Counter(df['Size'])))

128


In [53]:
from collections import Counter
print(len(Counter(df['ByteOffset_Delta_Class_1001'])))

1001


In [54]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size'].tolist()
size_test = df[training_pt_1+1:]['Size'].tolist()


In [55]:
lba_train= np.array(lba_train).reshape(-1,1)
lba_test= np.array(lba_test).reshape(-1,1)
size_train= np.array(size_train).reshape(-1,1)
size_test= np.array(size_test).reshape(-1,1)

In [56]:
def create_dataset2(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - 2 * window_size):
        a = dataset[i:(i + window_size), 0]
        #print(a)
        dataX.append(a)
        b = dataset[(i + window_size):(i + 2* window_size), 0]
        #print(b)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

lstm_num_timesteps = 32
    
X_train_lba, y_train_lba = create_dataset2(lba_train, lstm_num_timesteps)
X_test_lba, y_test_lba = create_dataset2(lba_test, lstm_num_timesteps)
X_train_size, y_train_size = create_dataset2(size_train, lstm_num_timesteps)
X_test_size, y_test_size = create_dataset2(size_test, lstm_num_timesteps)

In [57]:
X_train_lba.shape, y_train_lba.shape, X_test_lba.shape,  y_test_lba.shape

((34309601, 32), (34309601, 32), (11436490, 32), (11436490, 32))

In [58]:
X_train_size.shape, y_train_size.shape, X_test_size.shape,  y_test_size.shape

((34309601, 32), (34309601, 32), (11436490, 32), (11436490, 32))

In [59]:
lstm_num_features = 1
lstm_predict_sequences = True
lstm_num_predictions = 32


# X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
# X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))
    

y_train_lba = np.reshape(y_train_lba, (y_train_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_test_lba = np.reshape(y_test_lba, (y_test_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_train_size = np.reshape(y_train_size, (y_train_size.shape[0], lstm_num_predictions, lstm_num_features))
y_test_size = np.reshape(y_test_size, (y_test_size.shape[0], lstm_num_predictions, lstm_num_features))                        


In [ ]:
# Two classification outputs
import keras

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf






from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate , Dot
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, Reshape




# no_docs = len(y_train_lba)
maxlen= 32



# # define two sets of inputs
# inputA = Input(shape=(32,))
# inputB = Input(shape=(32,))
# # inputA = Sequential()
# # inputB = Sequential()
vocabulary_1 = len(Counter(df['ByteOffset_Delta_Class_1001']))


hidden_size = 200

# input=Input(shape=(no_docs,maxlen),dtype='float64')
inputA=Input(shape=(maxlen,),dtype='float64')  
inputB=Input(shape=(maxlen,),dtype='float64') 


# the first branch operates on the first input
x = Embedding(input_dim=vocabulary_1,output_dim=hidden_size,input_length=maxlen)(inputA)
x = Model(inputs=inputA, outputs=x)

# # the second branch opreates on the second input
y = Embedding(input_dim=vocabulary_2,output_dim=hidden_size,input_length=maxlen)(inputB)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = keras.layers.concatenate([x.output, y.output])

lstm1 = LSTM(hidden_size,return_sequences=True)(combined)
lstm2 = LSTM(hidden_size, return_sequences=True)(lstm1)

# create classification output
offset = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_1, activation='softmax'), name='offset')(lstm2)
iosize = keras.layers.wrappers.TimeDistributed(Dense(units=1, activation='sigmoid'), name='iosize')(lstm2)




model =Model([inputA,inputB],[offset,iosize]) # combining all into a Keras model

model.compile(optimizer='rmsprop',
              loss={'offset': 'sparse_categorical_crossentropy', 'iosize': 'mse'},
              loss_weights={'offset': 2., 'iosize': 1.5},
              metrics={ 'offset': 'categorical_accuracy', 'iosize': 'mae'})
model.summary()


In [ ]:
import time
num_epochs = 50
batch_size = 32


monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=1, save_best_only=True) # save best model


print('Train...')
start_time = time.time()

model.fit([X_train_lba,X_train_size],[y_train_lba,y_train_size],
          verbose=1,epochs=75,callbacks=[monitor,checkpointer])

# model.load_weights('best_weights.hdf5') # load weights from best model
print('Done ...!')

#model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)


Train...


/soe/hlitz/miniconda3/envs/py3_tf_gpu/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/75
 9518080/34309601 [=======>......................] - ETA: 21:01:24 - loss: 3.5675 - offset_loss: 1.7264 - iosize_loss: 0.0765 - offset_categorical_accuracy: 0.9661 - iosize_mae: 0.1993

In [ ]:
pred1,pred2 = model.predict([x_test_lba,x_test_size],verbose =1 )
# pred_1 = np.argmax(pred1,axis=1)
# pred_2 = np.argmax(pred2,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
max_lba_accuracy = 0
max_lba_accuracy_pos = 0

max_size_accuracy = 0
max_size_accuracy_pos = 0

import time
start_time = time.time()

for i in range(32):
    pred_1 = pred1[:,i,:]
    pred_2 = pred2[:,i,:]
    pred_1 = np.argmax(pred_1, axis=1)
    pred_2 = np.argmax(pred_2, axis=1)
    
    
    tmp_lba = accuracy_score(lba_test_final, pred_1)
    if (tmp_lba > max_lba_accuracy):
        max_lba_accuracy = tmp_lba
        max_lba_accuracy_pos = i
        
    tmp_size = accuracy_score(lba_size_final, pred_2)
    if (tmp_size > max_size_accuracy):
        max_size_accuracy = tmp_size
        max_size_accuracy_pos = i
    

print("Best IO Size Accuracy", str(max_size_accuracy))
print("Best IO Size Pos", str(max_size_accuracy_pos))

print("Best LBA Delta Accuracy", str(max_lba_accuracy))
print("Best LBA Delta Position", str(max_lba_accuracy_pos))

print("Elapsed time: {}".format((elapsed_time)))

In [ ]:
print("Done...!!")